### This notebook will be used for the Capstone Project.

In [2]:
import pandas as pd
import numpy as np
!pip install bs4
!pip install lxml
from bs4 import BeautifulSoup
import requests

print("Hello Capstone Project Course")

Hello Capstone Project Course


In [47]:
# getting data from internet
wikipedia_link=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# using beautiful soup to parse the HTML/XML.
soup = BeautifulSoup(wikipedia_link,'xml')
#print(soup.prettify())

In [111]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')           
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):            
            continue        
                     
    try:
        if ((tag_a_Borough is None)):            
            continue            
            
    except:        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):        
        continue                  
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [112]:
# Unique postcode
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

# Multi neighborhoods of one Postcode shown in one cell with comma. Not assigned Neighborhood assigned with the name of Borough 
for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 100


In [115]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M9P,Etobicoke,Westmount
1,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
2,M1H,Scarborough,Cedarbrae
3,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [116]:
df_toronto.shape

(100, 3)

In [117]:
address=pd.read_csv("https://cocl.us/Geospatial_data")
address.rename(columns={'Postal Code':'Postcode'}, inplace=True)
address.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [120]:
merged_df = pd.merge(df_toronto, address, left_on='Postcode', right_on='Postcode', how='outer')
merged_df.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9P,Etobicoke,Westmount,43.696319,-79.532242
1,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
2,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
3,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
5,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
6,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
7,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
8,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
9,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
